In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# 测试Embedding

In [2]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
print(lookup_tensor)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([0])
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


# N-gram

In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [4]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        #整段文字embeding长度
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [10]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        print('input------>',context_idxs)
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        print('target------>',[word_to_ix[target]])
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

input------> tensor([96, 91])
target------> [33]
input------> tensor([91, 33])
target------> [47]
input------> tensor([33, 47])
target------> [12]
input------> tensor([47, 12])
target------> [13]
input------> tensor([12, 13])
target------> [32]
input------> tensor([13, 32])
target------> [5]
input------> tensor([32,  5])
target------> [80]
input------> tensor([ 5, 80])
target------> [49]
input------> tensor([80, 49])
target------> [77]
input------> tensor([49, 77])
target------> [86]
input------> tensor([77, 86])
target------> [13]
input------> tensor([86, 13])
target------> [95]
input------> tensor([13, 95])
target------> [1]
input------> tensor([95,  1])
target------> [93]
input------> tensor([ 1, 93])
target------> [55]
input------> tensor([93, 55])
target------> [7]
input------> tensor([55,  7])
target------> [34]
input------> tensor([ 7, 34])
target------> [27]
input------> tensor([34, 27])
target------> [79]
input------> tensor([27, 79])
target------> [15]
input------> tensor([79

target------> [29]
input------> tensor([ 6, 29])
target------> [74]
input------> tensor([29, 74])
target------> [67]
input------> tensor([74, 67])
target------> [62]
input------> tensor([67, 62])
target------> [53]
input------> tensor([62, 53])
target------> [71]
input------> tensor([53, 71])
target------> [66]
input------> tensor([71, 66])
target------> [31]
input------> tensor([66, 31])
target------> [23]
input------> tensor([31, 23])
target------> [50]
input------> tensor([23, 50])
target------> [76]
input------> tensor([50, 76])
target------> [9]
input------> tensor([76,  9])
target------> [13]
input------> tensor([ 9, 13])
target------> [95]
input------> tensor([13, 95])
target------> [68]
input------> tensor([95, 68])
target------> [35]
input------> tensor([68, 35])
target------> [43]
input------> tensor([35, 43])
target------> [75]
input------> tensor([43, 75])
target------> [39]
input------> tensor([75, 39])
target------> [51]
input------> tensor([39, 51])
target------> [14]
in

input------> tensor([26, 82])
target------> [43]
input------> tensor([82, 43])
target------> [25]
input------> tensor([43, 25])
target------> [22]
input------> tensor([25, 22])
target------> [4]
input------> tensor([96, 91])
target------> [33]
input------> tensor([91, 33])
target------> [47]
input------> tensor([33, 47])
target------> [12]
input------> tensor([47, 12])
target------> [13]
input------> tensor([12, 13])
target------> [32]
input------> tensor([13, 32])
target------> [5]
input------> tensor([32,  5])
target------> [80]
input------> tensor([ 5, 80])
target------> [49]
input------> tensor([80, 49])
target------> [77]
input------> tensor([49, 77])
target------> [86]
input------> tensor([77, 86])
target------> [13]
input------> tensor([86, 13])
target------> [95]
input------> tensor([13, 95])
target------> [1]
input------> tensor([95,  1])
target------> [93]
input------> tensor([ 1, 93])
target------> [55]
input------> tensor([93, 55])
target------> [7]
input------> tensor([55,

target------> [94]
input------> tensor([83, 94])
target------> [49]
input------> tensor([94, 49])
target------> [16]
input------> tensor([49, 16])
target------> [6]
input------> tensor([16,  6])
target------> [29]
input------> tensor([ 6, 29])
target------> [74]
input------> tensor([29, 74])
target------> [67]
input------> tensor([74, 67])
target------> [62]
input------> tensor([67, 62])
target------> [53]
input------> tensor([62, 53])
target------> [71]
input------> tensor([53, 71])
target------> [66]
input------> tensor([71, 66])
target------> [31]
input------> tensor([66, 31])
target------> [23]
input------> tensor([31, 23])
target------> [50]
input------> tensor([23, 50])
target------> [76]
input------> tensor([50, 76])
target------> [9]
input------> tensor([76,  9])
target------> [13]
input------> tensor([ 9, 13])
target------> [95]
input------> tensor([13, 95])
target------> [68]
input------> tensor([95, 68])
target------> [35]
input------> tensor([68, 35])
target------> [43]
inp

input------> tensor([ 5, 59])
target------> [13]
input------> tensor([59, 13])
target------> [69]
input------> tensor([13, 69])
target------> [26]
input------> tensor([69, 26])
target------> [82]
input------> tensor([26, 82])
target------> [43]
input------> tensor([82, 43])
target------> [25]
input------> tensor([43, 25])
target------> [22]
input------> tensor([25, 22])
target------> [4]
input------> tensor([96, 91])
target------> [33]
input------> tensor([91, 33])
target------> [47]
input------> tensor([33, 47])
target------> [12]
input------> tensor([47, 12])
target------> [13]
input------> tensor([12, 13])
target------> [32]
input------> tensor([13, 32])
target------> [5]
input------> tensor([32,  5])
target------> [80]
input------> tensor([ 5, 80])
target------> [49]
input------> tensor([80, 49])
target------> [77]
input------> tensor([49, 77])
target------> [86]
input------> tensor([77, 86])
target------> [13]
input------> tensor([86, 13])
target------> [95]
input------> tensor([1

target------> [17]
input------> tensor([37, 17])
target------> [36]
input------> tensor([17, 36])
target------> [61]
input------> tensor([36, 61])
target------> [83]
input------> tensor([61, 83])
target------> [94]
input------> tensor([83, 94])
target------> [49]
input------> tensor([94, 49])
target------> [16]
input------> tensor([49, 16])
target------> [6]
input------> tensor([16,  6])
target------> [29]
input------> tensor([ 6, 29])
target------> [74]
input------> tensor([29, 74])
target------> [67]
input------> tensor([74, 67])
target------> [62]
input------> tensor([67, 62])
target------> [53]
input------> tensor([62, 53])
target------> [71]
input------> tensor([53, 71])
target------> [66]
input------> tensor([71, 66])
target------> [31]
input------> tensor([66, 31])
target------> [23]
input------> tensor([31, 23])
target------> [50]
input------> tensor([23, 50])
target------> [76]
input------> tensor([50, 76])
target------> [9]
input------> tensor([76,  9])
target------> [13]
inp

input------> tensor([80, 49])
target------> [77]
input------> tensor([49, 77])
target------> [86]
input------> tensor([77, 86])
target------> [13]
input------> tensor([86, 13])
target------> [95]
input------> tensor([13, 95])
target------> [1]
input------> tensor([95,  1])
target------> [93]
input------> tensor([ 1, 93])
target------> [55]
input------> tensor([93, 55])
target------> [7]
input------> tensor([55,  7])
target------> [34]
input------> tensor([ 7, 34])
target------> [27]
input------> tensor([34, 27])
target------> [79]
input------> tensor([27, 79])
target------> [15]
input------> tensor([79, 15])
target------> [65]
input------> tensor([15, 65])
target------> [60]
input------> tensor([65, 60])
target------> [8]
input------> tensor([60,  8])
target------> [70]
input------> tensor([ 8, 70])
target------> [63]
input------> tensor([70, 63])
target------> [48]
input------> tensor([63, 48])
target------> [41]
input------> tensor([48, 41])
target------> [45]
input------> tensor([41

In [11]:
for name, param in model.named_parameters():
    print(name,param.size())

embeddings.weight torch.Size([97, 10])
linear1.weight torch.Size([128, 20])
linear1.bias torch.Size([128])
linear2.weight torch.Size([97, 128])
linear2.bias torch.Size([97])
